In [ ]:
!pip install -U "transformers[torch]" datasets accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

model_name = "sshleifer/tiny-gpt2"   # 玩具模型，方便测试
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 随便加载一个小数据集
dataset = load_dataset("stas/openwebtext-10k", split="train[:100]")

print(dataset[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/951 [00:00<?, ?B/s]

openwebtext-10k.py: 0.00B [00:00, ?B/s]

RuntimeError: Dataset scripts are no longer supported, but found openwebtext-10k.py

In [ ]:
from datasets import load_dataset

dataset = load_dataset("NeelNanda/pile-10k", split="train[:100]")
print(dataset[0])


README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

data/train-00000-of-00001-4746b8785c874c(…):   0%|          | 0.00/33.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

In [1]:
%%capture
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl transformers accelerate datasets bitsandbytes peft


In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

# 1) 一些配置
max_seq_length = 2048        # 上下文长度
dtype = None                 # 让 unsloth 自动选 fp16 / bf16
load_in_4bit = True          # 4bit 量化，省显存

# 2) 选模型：1B 或 3B
model_name = "unsloth/Llama-3.2-3B-Instruct"   # 想换 3B 就改成 3B 的名字

# 3) 从 Hugging Face 加载模型 + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = model_name,
    max_seq_length  = max_seq_length,
    dtype           = dtype,
    load_in_4bit    = load_in_4bit,
    # 如果模型是 gated，可能需要：token="你的HF_TOKEN"
)

# 4) 给模型加 LoRA 适配层（PEFT）
model = FastLanguageModel.get_peft_model(
    model,
    r                         = 16,   # LoRA rank，越大越耗显存
    lora_alpha                = 16,
    lora_dropout              = 0,
    target_modules            = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "up_proj", "down_proj",
    ],
    use_gradient_checkpointing = "unsloth",
    random_state              = 3407,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


In [3]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# 1) 加载 FineTome-100k
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

# 2) 标准化成 role/content 格式
dataset = standardize_sharegpt(dataset)

# 3) 给 tokenizer 加上 Llama 3.1/3.2 的聊天模板
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# 4) 把 conversations 转成一整段 text
def format_prompts(examples):
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
        )
        for convo in examples["conversations"]
    ]
    return {"text": texts}

dataset = dataset.map(format_prompts, batched=True)
print(dataset[0]["text"][:500])


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluatio


In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

max_seq_length = 2048

# 为了先把流程跑通，只用前 2000 条样本
train_dataset = dataset.select(range(2000))

trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = train_dataset,
    dataset_text_field = "text",
    max_seq_length     = max_seq_length,
    data_collator      = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps                = 5,
        max_steps                   = 50,       # 先训练 50 步试流程
        learning_rate               = 2e-4,
        fp16                        = not is_bfloat16_supported(),
        bf16                        = is_bfloat16_supported(),
        logging_steps               = 1,
        output_dir                  = "outputs",
        report_to                   = "none",
        optim                       = "adamw_8bit",
        weight_decay                = 0.01,
        save_steps                  = 10,       # 每 10 步存一个 checkpoint
    ),
)

# 只在 assistant 回复部分算 loss（可选，但推荐）
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part    = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer.train()

# 训练完保存 LoRA 权重和 tokenizer
trainer.save_model("llama32-3b-finetome-lora")
tokenizer.save_pretrained("llama32-3b-finetome-lora")


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,267,584 of 3,232,017,408 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.066500
2,0.769200
3,0.705400
4,1.072700
5,0.644100
6,0.827500
7,0.877700
8,1.099700
9,0.796400
10,0.783300


('llama32-3b-finetome-lora/tokenizer_config.json',
 'llama32-3b-finetome-lora/special_tokens_map.json',
 'llama32-3b-finetome-lora/chat_template.jinja',
 'llama32-3b-finetome-lora/tokenizer.json')

In [5]:
from unsloth import FastLanguageModel
import torch

# 让模型进入推理模式（关闭 dropout 等，并用 Unsloth 的 2x 加速）
FastLanguageModel.for_inference(model)

def chat_once(user_prompt, system_prompt="You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]

    # 把对话转换成 Llama chat 模板的纯文本
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,  # 让模型知道该轮到 assistant 说话了
    )

    inputs = tokenizer(
        text,
        return_tensors="pt",
    ).to("cuda")   # 在 Colab GPU 上推理

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    # 只取新生成的部分
    generated_ids = outputs[0, inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(generated_ids, skip_special_tokens=True)
    print(answer)

# 试一条和训练集风格类似的问题
chat_once("用小学生能懂的方式解释一下布尔运算符是什么？")


布尔运算符是数学运算中的一个特殊类型，用于操作布尔值（真或假）。它主要包括两个运算：“和”（AND）和“或”（OR）。 

1. 布尔和（AND）运算：当两个布尔值都为真时，布尔和运算的结果为真。否则，结果为假。

2. 布尔或（OR）运算：当至少有一个布尔值为真时，布尔或运算的结果为真。否则，结果为假。

布尔运算符是数学中的一个基本概念，对于程序设计和逻辑表达式的分析非常重要。


In [6]:
!ls


huggingface_tokenizers_cache  outputs	   unsloth_compiled_cache
llama32-3b-finetome-lora      sample_data


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
# 在云盘里给这个 lab 建一个文件夹
#!mkdir -p "/content/drive/MyDrive/id2223_lab2"

# 把当前训练出来的 checkpoint 和 LoRA 目录都拷过去
!cp -r outputs "/content/drive/MyDrive/id2223_lab2/outputs_step50_3B"
!cp -r llama32-3b-finetome-lora "/content/drive/MyDrive/id2223_lab2/llama32-3b-finetome-lora"

# 看看里面有什么
!ls "/content/drive/MyDrive/id2223_lab2"


llama32-1b-finetome-lora      llama32-3b-finetome-lora	outputs_step50_3B
llama32-1b-finetome-merged16  outputs_step50


In [1]:
%%capture
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl transformers accelerate datasets bitsandbytes peft


In [12]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/MyDrive/id2223_lab2"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
llama32-1b-finetome-lora      llama32-3b-finetome-lora	outputs_step50_3B
llama32-1b-finetome-merged16  outputs_step50


In [13]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from trl import SFTTrainer

max_seq_length = 2048
dtype = None
load_in_4bit = True

adapter_dir = "/content/drive/MyDrive/id2223_lab2/llama32-3b-finetome-lora"
ckpt_dir    = "/content/drive/MyDrive/id2223_lab2/outputs_step50_3B"

# 1) 从云盘里的 LoRA 目录加载模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = adapter_dir,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)

# 2) 重新构建 FineTome 数据集（跟第一次训练时一样）
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
dataset = standardize_sharegpt(dataset)
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

def format_prompts(examples):
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
        )
        for convo in examples["conversations"]
    ]
    return {"text": texts}

dataset = dataset.map(format_prompts, batched=True)
train_dataset = dataset.select(range(2000))   # 先继续用同样的 2000 条

# 3) 定义 Trainer，这次想把训练步数拉到 100 步
trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = train_dataset,
    dataset_text_field = "text",
    max_seq_length     = max_seq_length,
    data_collator      = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps                = 5,
        max_steps                   = 100,      # <== 之前是 50，这里改成 100
        learning_rate               = 2e-4,
        fp16                        = not is_bfloat16_supported(),
        bf16                        = is_bfloat16_supported(),
        logging_steps               = 5,
        output_dir                  = ckpt_dir, # <== 直接写到云盘里的 outputs_step50
        report_to                   = "none",
        optim                       = "adamw_8bit",
        weight_decay                = 0.01,
        save_steps                  = 10,
    ),
)

# 4) 关键：从 step 50 的 checkpoint 继续训练
trainer.train(resume_from_checkpoint=f"{ckpt_dir}/checkpoint-50")

# 5) 训练完再把 LoRA+tokenizer 保存回同一个目录（覆盖原来的）
trainer.save_model(adapter_dir)
tokenizer.save_pretrained(adapter_dir)


==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,267,584 of 3,232,017,408 (0.60% trained)
	logging_steps: 5 (from args) != 1 (from trainer_state.json)


Step,Training Loss
51,1.263700
52,1.197700
53,1.385800
54,1.347400
55,1.030800
56,1.166600
57,0.950800
58,1.248100
59,0.957200
60,0.965400


('/content/drive/MyDrive/id2223_lab2/llama32-3b-finetome-lora/tokenizer_config.json',
 '/content/drive/MyDrive/id2223_lab2/llama32-3b-finetome-lora/special_tokens_map.json',
 '/content/drive/MyDrive/id2223_lab2/llama32-3b-finetome-lora/chat_template.jinja',
 '/content/drive/MyDrive/id2223_lab2/llama32-3b-finetome-lora/tokenizer.json')

In [15]:
!ls "/content/drive/MyDrive/id2223_lab2/outputs_step50_3B"


checkpoint-10	checkpoint-30  checkpoint-60  checkpoint-90
checkpoint-100	checkpoint-40  checkpoint-70  outputs
checkpoint-20	checkpoint-50  checkpoint-80  README.md


In [16]:
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)

def chat_once(user_prompt, system_prompt="You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    generated_ids = outputs[0, inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(generated_ids, skip_special_tokens=True)
    print(answer)

chat_once("再用小学生能懂的方式解释一下布尔运算符是什么？")


布尔运算符是数学中用于处理真假的操作。它有两个主要运算：AND（与）和 OR（或）。它们是用来判断一个表达式的真值。一个表达式是真值的，当它为 true 时，表达式就是 true。反之，一旦它为 false，表达式就是 false。

在布尔运算中，AND 代表“和”，它与两个或多个条件相关。只有当所有条件都为 true 时，AND 才会为 true。例如，在条件 a AND b 里面，b 只要为 false，a 就不会为 true。

OR 代表“或”，它与一个或多个条件相关。只要有一个条件为 true，OR 就会为 true。例如，在条件 a OR b 里面，a 只要为 true，b 就不会影响结果。

在布尔运算中，你可以使用这些运算符来判断一个表达式的真值。


In [17]:
# 把 LoRA 合并成一个完整模型，存在本地
merged_dir = "/content/llama32-3b-finetome-merged16"

model.save_pretrained_merged(
    merged_dir,
    tokenizer,
    save_method = "merged_16bit",   # 得到一个 FP16 的完整模型
)

!ls "$merged_dir"


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [03:27<00:00, 103.80s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [01:49<00:00, 54.76s/it]


Unsloth: Merge process complete. Saved to `/content/llama32-3b-finetome-merged16`
chat_template.jinja		  model.safetensors.index.json
config.json			  special_tokens_map.json
model-00001-of-00002.safetensors  tokenizer_config.json
model-00002-of-00002.safetensors  tokenizer.json


In [18]:
!cp -r "/content/llama32-3b-finetome-merged16" "/content/drive/MyDrive/id2223_lab2/"


In [19]:
from huggingface_hub import login
login()   # 粘贴 token 回车


In [20]:
HF_USERNAME = "yunquan01"   # 比如 'yunquan'，在 HF 右上角头像旁边那个名字

LORA_REPO   = f"{HF_USERNAME}/llama32-3b-finetome-lora"
MERGED_REPO = f"{HF_USERNAME}/llama32-3b-finetome-merged16"
print(LORA_REPO, MERGED_REPO)


yunquan01/llama32-3b-finetome-lora yunquan01/llama32-3b-finetome-merged16


In [21]:
from unsloth import FastLanguageModel

# （1）先上传 LoRA 版本（体积小）
model.push_to_hub(
    LORA_REPO,
    tokenizer,
)

# （2）再上传合并后的 16bit 模型（如果你之前已经 save_pretrained_merged）
model.push_to_hub_merged(
    MERGED_REPO,
    tokenizer,
    save_method = "merged_16bit",
)


Saved model to https://huggingface.co/yunquan01/llama32-3b-finetome-lora
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [03:30<00:00, 105.33s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [05:24<00:00, 162.46s/it]


Unsloth: Merge process complete. Saved to `/content/yunquan01/llama32-3b-finetome-merged16`


In [1]:
%%capture
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl transformers accelerate datasets bitsandbytes peft

from huggingface_hub import login
login()   # 粘贴你之前的 hf_token


In [22]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True

adapter_repo = "yunquan01/llama32-3b-finetome-lora"  # 你 LoRA 模型的 HF 仓库
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = adapter_repo,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)


==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
GGUF_REPO = "yunquan01/llama32-1b-finetome-gguf"   # 你想要的新仓库名

model.push_to_hub_gguf(
    GGUF_REPO,
    tokenizer,
    quantization_method = ["q4_k_m"],  # 经典 4-bit 量化，CPU 友好
)


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:28<00:00, 28.51s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:52<00:00, 52.33s/it]


Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_8seslxlc`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages


In [1]:
%%capture
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl transformers accelerate datasets bitsandbytes peft

from huggingface_hub import login
login()   # 粘贴你的 HF token


In [2]:
from unsloth import FastLanguageModel

max_seq_length = 1024      # 比之前 2048 小一点，省内存
dtype = None
load_in_4bit = True

adapter_repo = "yunquan01/llama32-1b-finetome-lora"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = adapter_repo,
    max_seq_length = max_seq_length,
    dtype          = dtype,
    load_in_4bit   = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 0 MLP layers.


In [ ]:
GGUF_REPO = "yunquan01/llama32-1b-finetome-gguf"

model.push_to_hub_gguf(
    GGUF_REPO,
    tokenizer,
    quantization_method = "q4_k_m",   # 或者换 "q4_0" 更轻一点
)


Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:32<00:00, 32.93s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:23<00:00, 23.88s/it]


Unsloth: Merge process complete. Saved to `/tmp/unsloth_gguf_2j21qxko`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: llama.cpp folder exists but binaries not found - will rebuild
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Install GGUF and other packages


In [23]:
!pip install -q transformers accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [25]:
MODEL_SMALL = "yunquan01/llama32-1b-finetome-merged16"
MODEL_BIG   = "yunquan01/llama32-3b-finetome-merged16"

def load_model(repo_id):
    tok = AutoTokenizer.from_pretrained(repo_id)
    mod = AutoModelForCausalLM.from_pretrained(
        repo_id,
        torch_dtype = torch.float16 if device == "cuda" else torch.float32,
    )
    mod.to(device)
    mod.eval()
    return tok, mod

small_tok, small_model = load_model(MODEL_SMALL)
big_tok,   big_model   = load_model(MODEL_BIG)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
import time

SYSTEM_PROMPT = (
    "You are a helpful assistant fine-tuned on the FineTome instruction dataset. "
    "Answer clearly and concisely."
)

def chat_once(tokenizer, model, user_message, max_new_tokens=128):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_message},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        t0 = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        dt = time.time() - t0

    gen_ids = outputs[0, inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return answer, dt


In [27]:
questions = {
    "Q1": "用小学生能懂的方式解释一下布尔运算符是什么？",
    "Q2": "“与（AND）” 和 “或（OR）” 在布尔运算里有什么区别？请用生活中的例子说明。",
    "Q3": "什么是“非（NOT）”运算？举两个例子。",
    "Q4": "在编程里，if (x > 3 && x < 10) 这句条件是什么意思？",
    "Q5": "在编程里，if (age < 18 || isStudent) 这句条件是什么意思？",
    # ... 可继续加到 Q10、Q15
}
results = {}

for qid, qtext in questions.items():
    print("="*80)
    print(qid, qtext)

    small_ans, small_t = chat_once(small_tok, small_model, qtext)
    big_ans,   big_t   = chat_once(big_tok,   big_model,   qtext)

    results[qid] = {
        "question": qtext,
        "small_answer": small_ans,
        "small_time": small_t,
        "big_answer": big_ans,
        "big_time": big_t,
    }

    print("\n[1B model answer] (time: %.2fs)\n" % small_t, small_ans)
    print("\n[3B model answer] (time: %.2fs)\n" % big_t, big_ans)


Q1 用小学生能懂的方式解释一下布尔运算符是什么？


AttributeError: 'LlamaAttention' object has no attribute 'apply_qkv'

In [1]:
!pip install -q transformers accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [2]:
MODEL_SMALL = "yunquan01/llama32-1b-finetome-merged16"
MODEL_BIG   = "yunquan01/llama32-3b-finetome-merged16"

def load_model(repo_id):
    tok = AutoTokenizer.from_pretrained(repo_id)
    mod = AutoModelForCausalLM.from_pretrained(
        repo_id,
        torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    )
    mod.to(device)
    mod.eval()
    return tok, mod

small_tok, small_model = load_model(MODEL_SMALL)
big_tok,   big_model   = load_model(MODEL_BIG)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import time

SYSTEM_PROMPT = (
    "You are a helpful assistant fine-tuned on the FineTome instruction dataset. "
    "Answer clearly and concisely."
)

def chat_once(tokenizer, model, user_message, max_new_tokens=128):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_message},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        t0 = time.time()
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )
        dt = time.time() - t0

    gen_ids = outputs[0, inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return answer, dt


In [4]:
questions = {
    "Q1": "用小学生能懂的方式解释一下布尔运算符是什么？",
    "Q2": "“与（AND）” 和 “或（OR）” 在布尔运算里有什么区别？请用生活中的例子说明。",
    "Q3": "什么是“非（NOT）”运算？举两个例子。",
    "Q4": "在编程里，if (x > 3 && x < 10) 这句条件是什么意思？",
    "Q5": "在编程里，if (age < 18 || isStudent) 这句条件是什么意思？",
    # ... 继续加 Q3 ~ Q10
}

results = {}

for qid, qtext in questions.items():
    print("="*80)
    print(qid, qtext)

    small_ans, small_t = chat_once(small_tok, small_model, qtext)
    big_ans,   big_t   = chat_once(big_tok,   big_model,   qtext)

    results[qid] = {
        "question": qtext,
        "small_answer": small_ans,
        "small_time": small_t,
        "big_answer": big_ans,
        "big_time": big_t,
    }

    print("\n[1B model answer] (time: %.2fs)\n" % small_t, small_ans)
    print("\n[3B model answer] (time: %.2fs)\n" % big_t, big_ans)


Q1 用小学生能懂的方式解释一下布尔运算符是什么？

[1B model answer] (time: 4.86s)
 布尔运算符是用来表示"布尔值"的符号。布尔值是表示一个真值或假值的符号，例如：T（真）和 F（假）。布尔运算符是用来在布尔值之间进行运算的符号，如：T + T、T - T、F + F、F - F。这些运算符将布尔值相加或相减，得到布尔值。

[3B model answer] (time: 2.61s)
 布尔运算符是用于判断真假问题的。比如，当你问“今天天气是否好”时，答案是“是”或者“否”，这就是布尔值。布尔值可以通过运算符进行比较，得到新的布尔值。
Q2 “与（AND）” 和 “或（OR）” 在布尔运算里有什么区别？请用生活中的例子说明。

[1B model answer] (time: 3.00s)
 与（AND）和或（OR）在布尔运算中都是用于检查条件的符号。在布尔运算中，与（AND）用来检查两个条件，若两者都为真，则结果为真；若两者都为假，则结果为假；若其中一个为真，另一个为假，则结果为真。或（OR）则用来检查一个条件，若条件为真，则结果为真；若条件为假，则结果为假。若其中一个条件为真，另一个条件为假，则结果为假。

[3B model answer] (time: 10.58s)
 与（AND）与或（OR）是布尔运算中两种基本运算。它们的区别在于它们的逻辑运算结果：

与（AND）：两个条件都需要满足，才会返回“true”。

或（OR）：只要有一个条件满足，或者两个条件同时满足，就会返回“true”。

生活中的例子：

1.  如果你有钱（A），你可以去看电影（B），这意味着你可以通过与运算（A & B）获得“true”。
2.  如果你有钱（
Q3 什么是“非（NOT）”运算？举两个例子。

[1B model answer] (time: 5.62s)
 非运算（NOT）是运算符之一，它运算时取反一个值，即 1 -> 0, 0 -> 1。

例子1： 
1 + NOT 2 = 1 + 0 = 1

例子2： 
5 + NOT 3 = 5 + 1 = 6

非运算在运算符与其前后都有一个值时最常见。它可以用来表示“非（NOT）”的概念。

[3B model answer] (time: 7.65s)
 “非（NOT）”运算是

In [5]:
# 把所有问题和回答保存成一个 txt 文件
with open("boolean_eval_results.txt", "w", encoding="utf-8") as f:
    for qid, r in results.items():
        f.write(f"{qid} {r['question']}\n")
        f.write(f"[1B model answer] (time: {r['small_time']:.2f}s)\n")
        f.write(r["small_answer"] + "\n\n")
        f.write(f"[3B model answer] (time: {r['big_time']:.2f}s)\n")
        f.write(r["big_answer"] + "\n")
        f.write("="*80 + "\n\n")


In [6]:
from google.colab import files
files.download("boolean_eval_results.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import time
SYSTEM_PROMPT=("You are a good model"
"Answer clearly and concisely"
)
model = "yunquan01/llama32-1b-finetome-merged16"


questions = {
    "Q1": "用小学生能懂的方式解释一下布尔运算符是什么？",
    "Q2": "“与（AND）” 和 “或（OR）” 在布尔运算里有什么区别？请用生活中的例子说明。",
    "Q3": "什么是“非（NOT）”运算？举两个例子。",
    "Q4": "在编程里，if (x > 3 && x < 10) 这句条件是什么意思？",
    "Q5": "在编程里，if (age < 18 || isStudent) 这句条件是什么意思？",
    # ... 继续加 Q3 ~ Q10
}
def load_model(repo_id):
    tok = AutoTokenizer.from_pretrained(repo_id)
    mod = AutoModelForCausalLM.from_pretrained(
        repo_id,
        torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    )
    mod.to(device)
    mod.eval()
    return tok, mod

tok, model = load_model(model)
def chat_once_with_params(tokenizer, model, user_message,
                          temperature, top_p, max_new_tokens=128):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_message},
    ]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True,
    )
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )

    gen_ids = outputs[0, inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return answer
qs = [questions["Q1"], questions["Q2"], questions["Q3"]]  # 你写成自己的问题字符串


for q in qs:
    ans_A = chat_once_with_params(tok, model, q, temperature=0.2, top_p=0.7)
    ans_B = chat_once_with_params(tok, model, q, temperature=0.7, top_p=0.9)

    print("Question:", q)
    print("[Setting A] temp=0.2, top_p=0.7\n", ans_A, "\n")
    print("[Setting B] temp=0.7, top_p=0.9\n", ans_B)
    print("="*80)

Question: 用小学生能懂的方式解释一下布尔运算符是什么？
[Setting A] temp=0.2, top_p=0.7
 布尔运算符是用来表示“等于”和“不等于”的符号。例如，等于运算符“=”，不等于运算符“≠”。 

[Setting B] temp=0.7, top_p=0.9
 布尔运算符是一种运算符，它用于表示“不”运算。它可以通过两个表达式进行运算来判断它们的结果。例如，如果两个表达式都是“0”，则结果为“0”，如果其中一个表达式是“1”则结果为“1”。布尔运算符可以用“!”（斜线）表示。例如，如果表达式“x>0”是真，表达式“x<0”是假，则布尔运算符“!x”将表达式“x>0”转换
Question: “与（AND）” 和 “或（OR）” 在布尔运算里有什么区别？请用生活中的例子说明。
[Setting A] temp=0.2, top_p=0.7
 “与（AND）”和“或（OR）”都是布尔运算的基本运算符，用于判断两个表达式的真实性。然而，它们在运算顺序上有所不同。

“与（AND）”运算是左向右的，意味着它会先检查第一个表达式，然后检查第二个表达式。如果第一个表达式为真，则第二个表达式也必须为真。否则，运算结果为假。

“或（OR）”运算是右向左的，意味着它会先检查 

[Setting B] temp=0.7, top_p=0.9
 与运算（AND）和或运算（OR）都是布尔运算的一部分，都是用于判断一个值的真（假）状态。然而，两者的区别在于它们的运算方法和结果。

1. 与运算（AND）是用括号内的表达式相乘，结果为真（假）时为真（假），否则为假（假）。因此，与运算是使用“（）”括号的。

2. or运算（OR）是用括号内的表达式相加，结果为真（假
Question: 什么是“非（NOT）”运算？举两个例子。
[Setting A] temp=0.2, top_p=0.7
 非运算（NOT）是运算符之一，它用于将一个值转换为其相反的值。例如：

1. NOT a = a → a = 0
2. NOT b = b → b = 1

在这些例子中，NOT a 和 NOT b 都是 a 和 b 的反值。 

[Setting B] temp=0.7, top_p=0.9
 非运算（NOT）是运算符之一，它表示与某个值相反。它可以应用于数值、标签

In [ ]:
MODEL_SMALL = "yunquan01/llama32-1b-finetome-merged16"
MODEL_BIG   = "yunquan01/llama32-3b-finetome-merged16"

def load_model(repo_id):
    tok = AutoTokenizer.from_pretrained(repo_id)
    mod = AutoModelForCausalLM.from_pretrained(
        repo_id,
        torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    )
    mod.to(device)
    mod.eval()
    return tok, mod

small_tok, small_model = load_model(MODEL_SMALL)
big_tok,   big_model   = load_model(MODEL_BIG)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]